In [1]:
import pandas as pd

In [2]:
train_df = pd.read_excel('Data_Train.xlsx')
test_df = pd.read_excel('Data_Test.xlsx')

## Analyzing data

In [3]:
train_df.head(10)

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,17.74
5,Hyundai EON LPG Era Plus Option,Hyderabad,2012,75000,LPG,Manual,First,21.1 km/kg,814 CC,55.2 bhp,5.0,2.35
6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,3.50
7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,Diesel,Automatic,First,11.36 kmpl,2755 CC,171.5 bhp,8.0,17.50
8,Volkswagen Vento Diesel Comfortline,Pune,2013,64430,Diesel,Manual,First,20.54 kmpl,1598 CC,103.6 bhp,5.0,5.20
9,Tata Indica Vista Quadrajet LS,Chennai,2012,65932,Diesel,Manual,Second,22.3 kmpl,1248 CC,74 bhp,5.0,1.95


In [4]:
train_df.dtypes

Name                  object
Location              object
Year                   int64
Kilometers_Driven      int64
Fuel_Type             object
Transmission          object
Owner_Type            object
Mileage               object
Engine                object
Power                 object
Seats                float64
Price                float64
dtype: object

In [5]:
train_df.isna().sum()

Name                  0
Location              0
Year                  0
Kilometers_Driven     0
Fuel_Type             0
Transmission          0
Owner_Type            0
Mileage               2
Engine               36
Power                36
Seats                42
Price                 0
dtype: int64

In [6]:
train_df.describe()

,Year,Kilometers_Driven,Seats,Price
count,6019.000000,6.019000e+03,5977.000000,6019.000000
mean,2013.358199,5.873838e+04,5.278735,9.479468
std,3.269742,9.126884e+04,0.808840,11.187917
min,1998.000000,1.710000e+02,0.000000,0.440000
25%,2011.000000,3.400000e+04,5.000000,3.500000
50%,2014.000000,5.300000e+04,5.000000,5.640000
75%,2016.000000,7.300000e+04,5.000000,9.950000
max,2019.000000,6.500000e+06,10.000000,160.000000


## Approach

In [7]:
# Splitting Training data(train_df) into train and test dfs to calculate accuracy. Will then use model on test_df

## Cleaning data 

In [8]:
def transform_owner(x):
    if x=='First':
        return 1
    elif x=='Second':
        return 2
    elif x=='Third':
        return 3
    return 4

In [9]:
def transform_transmission(x):
    if x=='Manual':
        return 1
    return 4

In [10]:
def cleanDF(df):
    # Remove extra characters like units and convert blank values to NAs.
    df['Engine']=df['Engine'].astype(str).str.replace('\D+', '')
    df['Engine']=pd.to_numeric(df['Engine'], errors='coerce')
    df['Power']=df['Power'].astype(str).str.replace('\D+', '')
    df['Power']=pd.to_numeric(df['Power'], errors='coerce')
    df['Mileage']=df['Mileage'].astype(str).str.replace('\D+', '')
    df['Mileage']=pd.to_numeric(df['Mileage'], errors='coerce',downcast='float')
    
    # If fuel type is CNG/LPG then units is km/kg. 1 kg roughly equals 1.5 litres. So dividing by 1.5 to convert km/kg to kml
    mask = (df['Fuel_Type'] == "CNG") | (df['Fuel_Type'] == "LPG")
    df['Mileage'][mask] = df['Mileage']/1.5
    
    #Remove NAS
    df = df.dropna()
    
    #Convert Categorical column-> Numerical Data
    df['Owner_Type']= df['Owner_Type'].apply(transform_owner)
    df['Transmission']= df['Transmission'].apply(transform_transmission)
    
    # Apply one hot encoding for Fuel Type
    a = pd.get_dummies(df['Fuel_Type'])
    df[['CNG','Diesel','LPG','Petrol']] = a
    df = df.drop(columns=['Fuel_Type'])
    
    return df

In [11]:
train_df = cleanDF(train_df)

/Users/manojkumar/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Users/manojkumar/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/manojkumar/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.ht

In [12]:
train_df.head()

,Name,Location,Year,Kilometers_Driven,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,CNG,Diesel,LPG,Petrol
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,1,1,177.333328,998.0,5816.0,5.0,1.75,1,0,0,0
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,1,1,1967.000000,1582.0,1262.0,5.0,12.50,0,1,0,0
2,Honda Jazz V,Chennai,2011,46000,1,1,182.000000,1199.0,887.0,5.0,4.50,0,0,0,1
3,Maruti Ertiga VDI,Chennai,2012,87000,1,1,2077.000000,1248.0,8876.0,7.0,6.00,0,1,0,0
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,4,2,152.000000,1968.0,1408.0,5.0,17.74,0,1,0,0


In [13]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [14]:
# Splitting cleaned data into X and Y
X = train_df[['Year','Kilometers_Driven','Transmission','Owner_Type','Mileage','Engine',
              'Power','Seats','CNG','Diesel','LPG','Petrol']]
Y= train_df['Price']

In [15]:
X.head()

,Year,Kilometers_Driven,Transmission,Owner_Type,Mileage,Engine,Power,Seats,CNG,Diesel,LPG,Petrol
0,2010,72000,1,1,177.333328,998.0,5816.0,5.0,1,0,0,0
1,2015,41000,1,1,1967.000000,1582.0,1262.0,5.0,0,1,0,0
2,2011,46000,1,1,182.000000,1199.0,887.0,5.0,0,0,0,1
3,2012,87000,1,1,2077.000000,1248.0,8876.0,7.0,0,1,0,0
4,2013,40670,4,2,152.000000,1968.0,1408.0,5.0,0,1,0,0


In [16]:
# Noramlising data
from sklearn import preprocessing
x = X.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
X = pd.DataFrame(x_scaled)

## Splitting train_df further into train and test so that we can calculate accuracy on test data

In [17]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)

### Linear Regression

In [18]:
regressor = LinearRegression()  
regressor.fit(x_train, y_train) #training the algorithm

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [19]:
Y_trainPredL = regressor.predict(x_test)

## Analyzing Results

In [20]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, Y_trainPredL))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, Y_trainPredL))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, Y_trainPredL)))

('Mean Absolute Error:', 4.175116959946918)
('Mean Squared Error:', 52.93714194533236)
('Root Mean Squared Error:', 7.27579149957806)


### K Means

In [21]:
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=2)
neigh.fit(x_train, y_train)
Y_trainPredK = neigh.predict(x_test)

## Analyzing Results

In [22]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, Y_trainPredK))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, Y_trainPredK))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, Y_trainPredK)))

('Mean Absolute Error:', 2.290136170212766)
('Mean Squared Error:', 36.79339161702128)
('Root Mean Squared Error:', 6.065755650949128)


### Decision Tree Regressor

In [23]:
from sklearn.tree import DecisionTreeRegressor
regr_1 = DecisionTreeRegressor(max_depth=2)
regr_1.fit(x_train, y_train)
Y_trainPredT = regr_1.predict(x_test)

In [24]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, Y_trainPredT))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, Y_trainPredT))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, Y_trainPredT)))

('Mean Absolute Error:', 4.218669593423504)
('Mean Squared Error:', 67.21095613494714)
('Root Mean Squared Error:', 8.198228841338057)


## Predict on unknown test_df

In [25]:
test_df= cleanDF(test_df)
X_test = test_df[['Year','Kilometers_Driven','Transmission','Owner_Type','Mileage','Engine',
              'Power','Seats','CNG','Diesel','LPG','Petrol']]
xt = X_test.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
X_test = pd.DataFrame(x_scaled)

/Users/manojkumar/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Users/manojkumar/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/manojkumar/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.ht

## Value predictions for test_df

In [26]:
y_pred_linear = regressor.predict(X_test) # using linear regression
y_pred_kmeans = neigh.predict(X_test) # using kmeans
y_pred_dtree = regr_1.predict(X_test) # using kmeans

In [27]:
y_pred_linear

array([-1.87633137e+00,  1.05091740e+01,  2.21746103e-01, ...,
        9.31709663e+00,  8.66380087e-04, -1.41194520e+00])

In [28]:
y_pred_kmeans

array([ 1.875, 12.2  ,  4.45 , ...,  4.2  ,  2.775,  2.45 ])

In [29]:
y_pred_dtree

array([ 3.87298163,  6.9363986 ,  3.87298163, ..., 10.0690287 ,
        3.87298163,  3.87298163])